In [1]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
import pickle

import torch
import torch.nn as nn
import numpy as np
import copy
import time
import torch.nn.functional as F
import torch.optim as optim
from itertools import repeat
from tqdm.notebook import tqdm

In [3]:
t1 = torch.rand(11, 1, 12, 13)
t2 = torch.rand(11, 12, 1, 13)
(t1 + t2).shape

torch.Size([11, 12, 12, 13])

In [3]:
import sys
sys.path.append('./SR-GNN-master/pytorch_code')

In [4]:
from model import *
from utils import *

In [5]:
class Opt():
    def __init__(self, ):
        self.dataset = 'yoochoose1_64'
        self.batchSize = 100
        self.hiddenSize = 100
        self.epoch = 30
        self.lr = 0.001
        self.lr_dc = 0.1
        self.lr_dc_step = 3
        self.l2 = 1e-5
        self.step = 1
        self.patience = 10
        self.nonhybrid = False
        self.validation = True
        self.valid_portion = 0.01
        
opt = Opt()

In [6]:
train_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/train.txt', 'rb'))
if opt.validation:
    train_data, valid_data = split_validation(train_data, opt.valid_portion)
    test_data = valid_data
else:
    test_data = pickle.load(open('./SR-GNN-master/datasets/' + opt.dataset + '/test.txt', 'rb'))

if opt.dataset == 'diginetica':
    n_node = 43098
elif opt.dataset == 'yoochoose1_64' or opt.dataset == 'yoochoose1_4':
    n_node = 37484
else:
    n_node = 310

In [7]:
set_ = set()
for i in train_data[0]:
    for j in i:
        set_.add(j)
for i in train_data[1]:
    set_.add(i)
item_num = len(set_)

item_mapping = {}
for i, item in enumerate(set_):
    item_mapping[item] = i

d_basket_train = {k: [item_mapping[item] for item in items + [train_data[1][k]]] for k, items in enumerate(train_data[0])}
d_basket_test = {k: [item_mapping[item] for item in items + [test_data[1][k]] if item in item_mapping] for k, items in enumerate(test_data[0])}
d_basket_test = {k: v for k, v in d_basket_test.items() if len(v) > 0}

In [8]:
class DataLoaderStoch():
    '''
    Класс автоматическрого создания батчей для трейна.
    '''
    def __init__(self, trans,
                 basket,
                 batchsize=128,
                 max_basket_size=23,
                 item_num=40,
                 shuffle=True):
        ''''
        На вход:
        trans          - список id чеков
        basket         - словарь товаров в коризне по чеку,
                         в виде списка из номеров товаров
        item_num  - кол-во товаров
        batchsize - размер батча
        shuffle   - перемешивать ли семплы
        На выход:
        батч в виде [(
                      корзины: list[list],
                      контекст: Tensor, size=(batchsize, context_dim),
                      таргетные продукты: LongTensor, size=(batchsize,),
                      клиенты: LongTensor, size=(batchsize,)
                     ),
                     таргет: Tensor, size=(batchsize,)]
        '''
        self.trans = trans
        self.basket = basket
        self.max_basket_size = max_basket_size
        self.batchsize = batchsize
        self.shuffle = shuffle
        self.prod_num = item_num
        
    def __iter__(self):  
        '''
        Метод вызывается при итерировании по объекту,
        например, через for.
        '''
        self.ids = set(range(len(self.trans)))
        return self._contaner_()
    
    def __len__(self):
        '''
        Возвращает кол-во батчей.
        '''
        batch_num = np.ceil(len(self.trans)/self.batchsize)
        return int(batch_num)
        
    def _contaner_(self):
        '''
        Метод берет подвыборку и формирует батч.
        '''
        while len(self.ids) != 0:
            if self.shuffle:
                size = min(len(self.ids), self.batchsize)
                idx_curr = np.random.choice(list(self.ids), size,
                                            replace=False)
            else:
                idx_curr = np.array(list(self.ids))[:self.batchsize]
            self.ids = self.ids.difference(idx_curr)
            yield self._make_sample_(self.trans[idx_curr])
        
    def foo(self, trans):
        '''
        Вспомогательная функция. см _make_sample_.
        '''
        trans_products = self.basket[trans]
        target_prod = trans_products[-1]
        without_target = trans_products[:-1]
        padding = [self.prod_num]*(self.max_basket_size - len(without_target))
        
        return padding+without_target, target_prod
        
    def _make_sample_(self, X):
        '''
        Метод возвращает готовый батч.
        '''
        self.X = X
        temp = list(map(self.foo, self.X))

        return [torch.LongTensor(np.array([row[0] for row in temp])),
                torch.LongTensor(np.array([row[1] for row in temp]))]

def generate_square_subsequent_mask(sz: int) -> torch.Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [9]:
#функция тестирования
def test(model, test_data):
    model.eval()
    hit, mrr = [], []
    batch_size = min(opt.batchSize, len(test_data[0]))
    for i in tqdm(range(0, len(test_data[0]), batch_size)):
        scores = model.predict(test_data[0][i:i+batch_size])
        sub_scores = np.array(scores.topk(20)[1])
        targets = test_data[1][i:i+batch_size]
        for score, target in zip(sub_scores, targets):
            target = model.item_mapping.get(target)
            hit.append(np.isin(target, score))
            if len(np.where(score == target)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr

#функция тестирования
def evaluate_net(net, testloader, use_cuda=True):
    net = net.eval()
    running_loss = 0.0
    # цикл по батчам внутри эпохи
    for i, data in enumerate(tqdm(testloader)):
        # берем очередной батч и его лейблы
        prods = data[0].to(device)
        labels = data[1].to(device)
        
        # получили выход сетки
        outputs = net(prods)
        
        # посчитали для этого выхода лосс
        loss = criterion(outputs, labels)
        
        #суммируемый лосс на обучении
        running_loss += float(loss)
        
    return running_loss/len(testloader)

#основная функция для обучения сети
def train_net(n_epochs, 
              net, 
              optimizer, 
              scheduler,
              criterion, 
              trainloader,
              testloader,
              test_data,
              prod_num,
              use_cuda=False,
             ):
    '''
    Функция обучения нейронной сети.
    На вход:
    n_epochs      - кол-во эпох
    net           - сеть для обучения
    optimizer     - оптимизатор для обучения
    criterion     - критерий оптимизации
    trainloader   - даталоадер для трейна
    testloader    - даталоадер для теста
    d_food_cost_idx - цены на товары
    prod_num      - кол-во продуктов
    use_cuda      - использовать ли cuda
    verbose       - если 0, то не выводит 
                    качество на валидации,
                    если > 0 выводит качество
                    на валлидации каждые verbose
                    эпох
    early_stopping_len - после какого кол-ва эпох
                    без улучшения качества 
                    надо прекратить обучение
    '''
    
    if use_cuda:
        net = net.cuda()
    
    test_loss = 0
    
    # основной цикл по всем эпохам
    for epoch in range(n_epochs):
        net = net.train()
    
        running_loss = 0.0
        # цикл по батчам внутри эпохи
        for i, data in enumerate(tqdm(trainloader)):
            # берем очередной батч и его лейблы
            prods = data[0].to(device)
            labels = data[1].to(device)

            # всегда перед вычислением градиентов зануляем их, чтобы не накапливались
            optimizer.zero_grad()

            # получили выход сетки
            outputs = net(prods)
            
            # посчитали для этого выхода лосс
            loss = criterion(outputs, labels)
            
            # вычислили градиенты loss по параметрам сети (w)
            loss.backward()
    
            #далем шаг по антиградиенту - обновляем веса сети
            optimizer.step()
            
            #суммируемый лосс на обучении
            running_loss += float(loss)
        
        scheduler.step()
        # валидируемся
        hit, mrr = test(net, test_data)
        test_loss = evaluate_net(net, testloader, use_cuda=use_cuda)
        
        # логируем после каждой эпохи        
        print('Epoch {}. \nTrain_loss: {:.6f}' .format(epoch + 1, running_loss / len(trainloader)))
        print(f'Test_loss: {test_loss}')
        print(f'Test eval: hit - {hit}, mrr - {mrr}')
        print('------------------------------')

    print('Finished Training')
    return net

In [10]:
import torch
import torch.nn as nn
import numpy as np
import copy
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils.extmath import randomized_svd

class Embedding_prod(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, prod_num, d, dropout=0.1):
        super(Embedding_prod, self).__init__()
        self.prod_embedd = nn.Embedding(prod_num+1, d)
        self.pos_encoder = PositionalEncoding(d, dropout)
        
    def forward(self, prods):
        prod_embedd = self.prod_embedd(prods)
        prod_embedd = self.pos_encoder(prod_embedd)
        return prod_embedd

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x.permute(1, 0, 2)
        x = x + self.pe[:x.size(0)]
        return self.dropout(x).permute(1, 0, 2)

class Embedding_transformer(nn.Module):
    '''
    Нейросеть для эмбеддинга товара.
    '''
    def __init__(self, d, layer_num=2, layer_type='transformer', dim_feedforward=2048, dropout=0.1, max_basket_size=146):
        '''
        На вход:
        X - матрица (товары, эмбеддинг)
        use_cuda - использовать ли cuda
        
        Forward принимает на вход или
        список, содержащий номера товаров,
        или просто номер товара.
        '''
        super(Embedding_transformer, self).__init__()
        self.layer_type = layer_type
        if layer_type == 'attention':
            self.transformer = nn.ModuleList([nn.MultiheadAttention(d, 
                                                                    num_heads=1,
                                                                    batch_first=True,
                                                                    )
                                          for _ in range(layer_num)])
        elif layer_type == 'transformer':
            encoder_layer = nn.TransformerEncoderLayer(d_model=d,
                                                       nhead=1,
                                                       dropout=dropout,
                                                       dim_feedforward=dim_feedforward,
                                                       batch_first=True,
                                                      )
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=layer_num)
        #self.transformer_layer = nn.TransformerEncoderLayer(d,
        #                                                   nhead=1,
        #                                                   batch_first=True)


    def forward(self, batch):
        #t = self.transformer(batch)
        t = batch
        if self.layer_type == 'attention':
            for layer in self.transformer:
                t_ = t
                t = layer(t, t, t)[0]
                t = t_ + t
        elif self.layer_type == 'transformer':
            t = self.transformer(t)
        return t

class Net(nn.Module):
    def __init__(self, products_embedd, transformer,
                 d, max_basket_size, prod_num, item_mapping=item_mapping):
        '''
        На вход:
        products_embedd - объект класса Embedding_prod
        '''
        super(Net, self).__init__()
        self.embedding = products_embedd
        self.transformer = transformer
        self.max_basket_size = max_basket_size
        self.linear1 = nn.Linear(max_basket_size, 1)
        self.linear2 = nn.Linear(d, prod_num)
        self.max_basket_size = max_basket_size
        self.prod_num = prod_num
        self.item_mapping = item_mapping
    
    def predict(self, baskets):
        rows = []
        for basket in baskets:
            basket_ = [self.item_mapping[item] for item in basket if item in item_mapping]
            padding = [self.prod_num]*(self.max_basket_size - len(basket_))
            rows.append(padding + basket_)
        return self.forward(torch.LongTensor(np.array(rows)))
    
    def forward(self, prods):
        # эмделлинг товара [batch_dim, max_basket_size, embedd_dim]
        embedd = self.embedding(prods)
        # пропускаем через череду attention
        embedd = self.transformer(embedd)
        # пропускаем чеоез линейныt слои
        embedd = embedd.permute(0, 2, 1)
        embedd = self.linear1(embedd).squeeze(2)
        return self.linear2(embedd)

In [11]:
max_basket_size = max(map(len, train_data[0]))+1
max_basket_size

146

In [12]:
device = 'cpu'

In [13]:
# создаем сеть
embedd_dim = 32 # 512
prod_embedd = Embedding_prod(item_num, embedd_dim)
transformer = Embedding_transformer(embedd_dim,
                                    layer_num=1,
                                    layer_type='attention',
                                    dim_feedforward=64,
                                    dropout=0.1,
                                    max_basket_size=max_basket_size)
net_model = Net(prod_embedd, transformer, embedd_dim, max_basket_size, item_num)
net_model.to(device)

Net(
  (embedding): Embedding_prod(
    (prod_embedd): Embedding(17353, 32)
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (transformer): Embedding_transformer(
    (transformer): ModuleList(
      (0): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
      )
    )
  )
  (linear1): Linear(in_features=146, out_features=1, bias=True)
  (linear2): Linear(in_features=32, out_features=17352, bias=True)
)

In [14]:
learning_rate= 1e-3 # 1e-4
optimizer = optim.Adam(net_model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8, verbose=True) # 0.8

# критерий оптимизации
criterion = nn.CrossEntropyLoss()

Adjusting learning rate of group 0 to 1.0000e-03.


In [15]:
torch.set_num_threads(32)

In [16]:
# создаем даталоадер для обучения
dataloader_train = DataLoaderStoch(np.array(list(d_basket_train.keys())),
                      d_basket_train,
                      max_basket_size=max_basket_size,
                      batchsize=64,
                      shuffle=True)

# создаем даталоадер для валидации
dataloader_test = DataLoaderStoch(np.array(list(d_basket_test.keys())),
                      d_basket_test,
                      max_basket_size=max_basket_size,
                      batchsize=64,
                      shuffle=True)

# учим сеть
net_model = train_net(50,
                   net_model,
                   optimizer,
                   scheduler,
                   criterion,
                   dataloader_train,
                   dataloader_test,
                   test_data,
                   item_num,
                   use_cuda=False,
                  )

  0%|          | 0/5722 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 8.0000e-04.


  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch 1. 
Train_loss: 6.983041
Test_loss: 6.011591335822796
Test eval: hit - 10.948905109489052, mrr - 3.1867496733764664
------------------------------


...

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch 3. 
Train_loss: 5.393295
Test_loss: 5.279505713232632
Test eval: hit - 14.165990808326576, mrr - 4.472298035233838
------------------------------


  0%|          | 0/5722 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
import torch
from torch import nn

In [7]:
criterion = nn.CrossEntropyLoss()

In [26]:
N = 10
C = 4
d = 20

target = torch.LongTensor(torch.randint(C, (N, d)))
scores = torch.zeros(N, C, d)
criterion(scores, target)

tensor(1.3863)

In [22]:
target

tensor([[2, 2, 0, 2, 1, 3, 1, 3, 1, 1, 1, 2, 0, 1, 3, 0, 0, 1, 1, 1],
        [3, 3, 3, 0, 3, 3, 1, 1, 2, 3, 3, 0, 0, 3, 1, 1, 3, 3, 0, 1],
        [0, 2, 1, 2, 0, 1, 1, 3, 2, 2, 1, 0, 0, 3, 1, 1, 0, 0, 0, 2],
        [0, 0, 1, 0, 3, 2, 0, 1, 1, 3, 2, 3, 1, 0, 0, 2, 3, 1, 0, 0],
        [1, 2, 0, 2, 1, 1, 0, 2, 2, 3, 2, 0, 0, 2, 0, 1, 1, 0, 1, 2],
        [3, 0, 3, 1, 1, 0, 2, 0, 3, 1, 1, 3, 2, 1, 3, 0, 1, 3, 1, 0],
        [2, 1, 3, 3, 2, 1, 1, 1, 2, 0, 2, 1, 2, 3, 1, 2, 0, 0, 0, 1],
        [1, 3, 3, 1, 3, 0, 0, 0, 2, 0, 1, 3, 1, 1, 2, 3, 0, 1, 3, 3],
        [2, 0, 1, 3, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 1, 0, 2, 1, 0, 0],
        [1, 3, 0, 3, 0, 1, 2, 0, 3, 1, 0, 0, 0, 2, 0, 2, 2, 1, 1, 1]])

In [2]:
import torch

In [19]:
transformer = torch.nn.Transformer(batch_first=True)

In [21]:
N = 12
C = 4
Lin = 10
Lout = 20
d = 512
input_ = torch.rand((N, Lin, d))
output_ = torch.rand((N, Lout, d))
transformer(input_, output_).shape

torch.Size([12, 20, 512])

In [14]:
input_.shape

torch.Size([10, 512])